# Introduction to Intake-ESM

**Max Grover**

**08 December 2021**

---

# Overview

1. What is Intake-ESM?
2. How can we use it to read in data?
3. How do we work with dictionaries of datasets?
4. How do I write my analysis to work with Intake-ESM?

# Prerequisites

|Concepts | Importance | Notes |
| --- | --- | --- |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/xarray/xarray.html) | Necessary | - |
| [Introduction to Matplotlib](https://foundations.projectpythia.org/core/matplotlib/matplotlib.html) | Helpful | - |

- Time to learn: 50 minutes.

---

# Setup

## Imports

In [ ]:
import intake
from distributed import Client, LocalCluster
import xarray as xr
import matplotlib.pyplot as plt
import fsspec

## Spin up a Dask Cluster

In [ ]:
cluster = LocalCluster()
client = Client(cluster)
client

### Investigate the files on AWS from the CESM-LENS Dataset

In [ ]:
fs = fsspec.filesystem('s3', anon=True)
bucket = 'ncar-cesm-lens'
fs.ls(bucket)

In [ ]:
bucket = 'ncar-cesm-lens/ocn'
fs.ls(bucket)

In [ ]:
bucket = 'ncar-cesm-lens/ocn/monthly'
fs.ls(bucket)

### Loading in Data Using Xarray

In [ ]:
var = 'T'
atmosphere_ds = xr.open_zarr(f's3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-{var}.zarr', storage_options={'anon':True})
atmosphere_ds

In [ ]:
variables = ['T', 'Q']
ds_list = []

# Loop through the different files and add them to the list of datasets
for var in variables:
    ds_list.append(xr.open_zarr(f's3://ncar-cesm-lens/atm/monthly/cesmLE-RCP85-{var}.zarr', storage_options={'anon':True}))
    
atmosphere_merged = xr.merge(ds_list)
atmosphere_merged

## Using Intake-ESM
We can use the Intake-ESM catalog from the CESM-LENS dataset (`https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/aws-cesm1-le.json`) to work with these data!

In [ ]:
data_catalog = intake.open_esm_datastore('https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/aws-cesm1-le.json')
data_catalog

In [ ]:
data_catalog.df

## search

In [ ]:
search1 = data_catalog.search(variable=['T','Q','TEMP','SALT'],frequency='monthly')
search1

In [ ]:
search1.df

## load the data

In [ ]:
dsets = search1.to_dataset_dict(storage_options={'anon':True})
dsets.keys()

In [ ]:
oceanCTL_ds = dsets['ocn.CTRL.monthly']
oceanCTL_ds

In [ ]:
temp = oceanCTL_ds.TEMP

In [ ]:
temp0 = oceanCTL_ds.TEMP.isel(time=0)

In [ ]:
SST0 = temp0.isel(z_t=0)

In [ ]:
SST0

In [ ]:
%%timeit
SST0 = SST0.compute()

In [ ]:
SST0.plot()

# The main steps

## 1. load in catalog

In [ ]:
data_catalog = intake.open_esm_datastore('https://raw.githubusercontent.com/NCAR/cesm-lens-aws/master/intake-catalogs/aws-cesm1-le.json')
data_catalog

In [ ]:
catalog_subset = data_catalog.search(component='atm',
                                     variable=['T','Q'],
                                     frequency='monthly',
                                     experiment=['RCP85','20C'])
catalog_subset.df

In [ ]:
catalog_subset.df.variable.unique()

In [ ]:
dsets2 = catalog_subset.to_dataset_dict(storage_options={'anon':True})
dsets2.keys()

In [ ]:
historical_ds = dsets2['atm.20C.monthly']
future_ds = dsets2['atm.RCP85.monthly']

In [ ]:
lat = 40.015
lon = 105.2705
variable = 'T'

historical_ds.isel(member_id=0,lev=-1).sel(lat=lat,lon=lon,method='nearest')

In [ ]:
%%time
output = historical_ds.isel(member_id=0,lev=-1).sel(lat=lat,lon=lon,method='nearest').isel(time=range(2)).T.compute()

In [ ]:
client.shutdown()